# 02: Real-World — Bounded Contexts and Multi-Instance Kinds

The `examples/shop/` fixture contains a clean-architecture e-commerce shop with domain entities, application ports/use-cases, and infrastructure adapters.

This notebook:
1. Writes Kind definitions for the shop codebase (single instance)
2. Restructures into two bounded contexts (payments + orders) sharing the same Kind type
3. Demonstrates that contracts are enforced across all instances

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

## Setup

In [ ]:
import { PROJECT_ROOT, ksc, tree, copyDir, copyFixture, writeFile, readFile, cleanup } from './lib.ts';
console.log("Setup complete.");

---

## Part 1: Single Instance — The Shop

We start with the shop fixture and write a single Kind definition for it.

In [ ]:
const DEMO = copyFixture("shop");

console.log("=== Project structure ===");
await tree(DEMO);

In [ ]:
writeFile(DEMO, "src/context.ts", `
import type { Kind, Instance } from 'kindscript';

type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;
type ApplicationLayer = Kind<"ApplicationLayer">;
type InfrastructureLayer = Kind<"InfrastructureLayer">;

type ShopContext = Kind<"ShopContext", {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [
    ["domain", "application"],
    ["domain", "infrastructure"],
    ["application", "infrastructure"],
  ];
}>;

export const shop = {
  domain: {},
  application: {},
  infrastructure: {},
} satisfies Instance<ShopContext, '.'>;
`);

console.log("Wrote src/context.ts — 3 noDependency pairs + purity on domain");

The `satisfies Instance<ShopContext, '.'>` declares a ShopContext instance located at `'.'` — the directory containing the declaration file (`src/`). Member names map to subdirectories:
- `domain` → `src/domain/`
- `application` → `src/application/`
- `infrastructure` → `src/infrastructure/`

`DomainLayer` has `{ pure: true }` — an intrinsic constraint that automatically generates purity enforcement when used as a member.

In [ ]:
await ksc("check", DEMO);

---

## Part 2: Restructure into Bounded Contexts

We want two bounded contexts — **orders** and **payments** — each with its own domain/application/infrastructure layers, sharing the same Kind definition.

We'll restructure the project: move the current `src/` content into `src/orders/`, and create a new `src/payments/` context.

In [ ]:
// Restructure: single-instance → two bounded contexts
// Move existing src/ layers into src/orders/, create src/payments/ with new code

const tmpSrc = `${DEMO}/_src_backup`;
Deno.renameSync(`${DEMO}/src`, tmpSrc);

// Orders context: copy existing layers
Deno.mkdirSync(`${DEMO}/src/orders`, { recursive: true });
copyDir(`${tmpSrc}/domain`, `${DEMO}/src/orders/domain`);
copyDir(`${tmpSrc}/application`, `${DEMO}/src/orders/application`);
Deno.mkdirSync(`${DEMO}/src/orders/infrastructure`, { recursive: true });
for (const f of ["sql-order-repository.ts", "in-memory-catalog.ts", "email-notification.ts"]) {
  Deno.copyFileSync(`${tmpSrc}/infrastructure/${f}`, `${DEMO}/src/orders/infrastructure/${f}`);
}

// Payments context: copy domain (shared value objects), add payment-specific code
copyDir(`${tmpSrc}/domain`, `${DEMO}/src/payments/domain`);

writeFile(DEMO, "src/payments/application/payment-gateway.port.ts", `
export interface PaymentGatewayPort {
  charge(customerId: string, amountCents: number): boolean;
  refund(transactionId: string): boolean;
}
`);

writeFile(DEMO, "src/payments/application/process-payment.ts", `
import { Money } from '../domain/money';
import { PaymentGatewayPort } from './payment-gateway.port';

export function processPayment(
  customerId: string,
  amount: Money,
  gateway: PaymentGatewayPort,
): boolean {
  return gateway.charge(customerId, Math.round(amount.amount * 100));
}
`);

writeFile(DEMO, "src/payments/infrastructure/stripe-gateway.ts", `
import { PaymentGatewayPort } from '../application/payment-gateway.port';

export class StripeGateway implements PaymentGatewayPort {
  charge(customerId: string, amountCents: number): boolean {
    console.log('[Stripe] Charging', customerId, amountCents, 'cents');
    return true;
  }
  refund(transactionId: string): boolean {
    console.log('[Stripe] Refunding', transactionId);
    return true;
  }
}
`);

Deno.removeSync(tmpSrc, { recursive: true });

console.log("=== Restructured project ===");
await tree(DEMO);

### Write definition files — one per bounded context

The shared `BoundedContext` Kind type is defined in each file. The `Instance<T, '.'>` path is relative to the declaration file, so each context's `'.'` resolves to its own directory.

In [ ]:
const sharedKindDefs = `
import type { Kind, Instance } from 'kindscript';

type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;
type ApplicationLayer = Kind<"ApplicationLayer">;
type InfrastructureLayer = Kind<"InfrastructureLayer">;

type BoundedContext = Kind<"BoundedContext", {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [
    ["domain", "application"],
    ["domain", "infrastructure"],
    ["application", "infrastructure"],
  ];
}>;
`.trimStart();

// Two definition files — same Kind, different locations
writeFile(DEMO, "src/payments/payments.ts", sharedKindDefs + `
// Location: '.' resolves to src/payments/ (this file's directory)
export const payments = {
  domain: {},
  application: {},
  infrastructure: {},
} satisfies Instance<BoundedContext, '.'>;
`);

writeFile(DEMO, "src/orders/orders.ts", sharedKindDefs + `
// Location: '.' resolves to src/orders/ (this file's directory)
export const orders = {
  domain: {},
  application: {},
  infrastructure: {},
} satisfies Instance<BoundedContext, '.'>;
`);

console.log("Wrote two definition files sharing BoundedContext Kind");
console.log("  src/payments/payments.ts  →  root: src/payments/");
console.log("  src/orders/orders.ts      →  root: src/orders/");

---

## Part 3: Check — Both Contexts

Both bounded contexts should satisfy all contracts.

In [ ]:
await ksc("check", DEMO);

All contracts pass. Each context is checked independently — violations in one don't affect the other.

---

## Part 4: Violations Are Caught in Both Contexts

### Purity violation in payments

The payments domain has clean entities. Let's break purity by adding a Node.js import.

In [ ]:
const cleanMoney = readFile(DEMO, "src/payments/domain/money.ts");

writeFile(DEMO, "src/payments/domain/money.ts",
  `import * as crypto from 'crypto';\n\n` + cleanMoney
);

console.log("=== Violation: payments domain imports 'crypto' ===");
await ksc("check", DEMO);

KindScript reports `KS70003` for the impure import. Both contexts share the same Kind type, and contracts are generated for *all* instances.

### noDependency violation in orders

Restore payments and break orders with a different violation type.

In [ ]:
// Restore payments
writeFile(DEMO, "src/payments/domain/money.ts", cleanMoney);

// Break orders: domain imports from infrastructure
const cleanOrder = readFile(DEMO, "src/orders/domain/order.ts");

writeFile(DEMO, "src/orders/domain/order.ts",
  `import { SqlOrderRepository } from '../infrastructure/sql-order-repository';\n\n` + cleanOrder
);

console.log("=== Violation: orders domain imports from infrastructure ===");
await ksc("check", DEMO);

In [ ]:
// Restore and verify clean
writeFile(DEMO, "src/orders/domain/order.ts", cleanOrder);

console.log("=== Restored — both contexts clean ===");
await ksc("check", DEMO);

---

## Design Discussion

### Explicit instance location

The second type parameter on `Instance<T, Path>` declares where the instance lives relative to the declaration file:
- `'.'` — current directory (most common)
- `'./subdirectory'` — a sibling subdirectory
- `'../other'` — a parent-relative path

This means you can define an instance **anywhere** and point it at **any** directory. The path works like a TypeScript import — relative to the file that declares it.

### Shared vs per-instance constraints

Constraints on the Kind type apply to **all** instances. Same Kind = same rules. If you need different rules per context, use separate Kind types.

### Cross-context dependencies

KindScript's constraints operate *within* a bounded context (e.g., payments.domain can't import from payments.infrastructure). Cross-context dependencies (payments importing from orders) aren't currently expressible. You'd need a higher-level Kind with bounded contexts as members.

In [ ]:
cleanup(DEMO);
console.log("Working directory cleaned up.");

---

## Summary

| Step | What we did |
|------|-------------|
| Single instance | Defined Kind types + constraints, verified all pass |
| Restructure | Split into orders/ and payments/ bounded contexts |
| Multi-instance | Shared `BoundedContext` Kind with separate definition files |
| Violations | Purity + noDependency violations caught in both contexts |

**Key takeaway:** The shared-Kind multi-instance pattern is the recommended approach for bounded contexts. Define the Kind type once, create a definition file per context. `Instance<T, '.'>` resolves to each file's own directory, so the same Kind maps to different locations automatically.

**Next:** See [03-typekind.ipynb](03-typekind.ipynb) for declaration-level enforcement, or [docs/06-tutorial.md](../docs/06-tutorial.md) for a complete static walkthrough including real-world design system modeling.